# VADER N-gram analysis Part 2 

*NOTE*: Notebooks may not run in order and processing may take quite some time to complete 

Final Processing and export for tableau 

In [12]:
import pandas as pd  
import numpy as np 
from tqdm import tqdm

In [3]:
df = pd.read_csv('NLP_VADER_sentiment_ngram.csv')

In [7]:
df.drop(columns=['Unnamed: 0'], inplace=True)

In [9]:
df[['postal_code', 'name-2', '3grams', '3grams_scores_VADER']].head()

,postal_code,name-2,3grams,3grams_scores_VADER
0,19135,services,"[('recently', 'cadillac', 'repaired'), ('cadil...","[0.0, 0.0, 0.0, 0.0, 0.6249, 0.6249, 0.6249, 0..."
1,19135,services,"[('lens', 'auto', 'body'), ('auto', 'body', 'w...","[0.0, -0.6249, -0.6249, -0.6249, 0.0, 0.0, 0.0..."
2,19135,services,"[('lens', 'auto', 'body'), ('auto', 'body', 'a...","[0.0, 0.6249, 0.6249, 0.6249, 0.0, 0.0, 0.0, 0..."
3,19135,services,"[('several', 'transaction', 'autobody'), ('tra...","[0.0, 0.0, 0.5719, 0.5719, 0.5719, 0.0, 0.6249..."
4,19135,services,"[('past', 'years', 'taken'), ('years', 'taken'...","[0.0, 0.0, 0.0, 0.0, -0.3612, -0.3612, -0.3612..."


In [10]:
df.shape

(970161, 9)

In [11]:
df2 = pd.read_csv('NLP_VADER_sentiment_ngrams_nonzero.csv')

In [17]:
df2.drop(columns=['Unnamed: 0'], inplace=True)

In [14]:
len(df2)

21344143

In [15]:
# Initialize tqdm for the concatenation process
concat_pbar = tqdm(total=2, desc="Concatenating DataFrames")

# Select top 3 and bottom 3 rows by score within each group
top_rows = df2.nlargest(2, 'score')
bottom_rows = df2.nsmallest(2, 'score')

# Concatenate DataFrames and update progress bar
result_df = pd.concat([top_rows, bottom_rows])
concat_pbar.update(1)

# Close tqdm progress bar for concatenation process
concat_pbar.close()

python(71211) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Concatenating DataFrames:  50%|█████▌     | 1/2 [00:25<00:25, 25.29s/it]


In [16]:
result_df

,Unnamed: 0,postal_code,category,business_id,preprocessed_text_v1,num_tokens,avg_sent_across_3gram_VADER,ngram_pair,score
12722963,12722963,19107,coffee,atZ_olNKXOG4rEr6mccN8g,beilers ily ily ily cant often great yelp worl...,58,0.287770,"('ily', 'ily', 'ily')",0.9349
288224,288224,19103,services,-cLmu1wRg6DXCTl_MDo8EA,search best pole studio philadelphia page foun...,201,0.238324,"('sweetheart', 'love', 'love')",0.9287
2516070,2516070,19107,services,6IG4SysBKyRnFW_e22q13A,hello there bring tonight th time happened pas...,158,-0.051028,"('raped', 'killed', 'worst')",-0.9349
2905279,2905279,19123,nightlife,73UjNbSoQjQAOS45rcihFg,stars bleepin yelp last night extremely hungry...,140,0.075346,"('evil', 'evil', 'evil')",-0.9349


In [19]:
df2.columns

Index(['postal_code', 'category', 'business_id', 'preprocessed_text_v1',
       'num_tokens', 'avg_sent_across_3gram_VADER', 'ngram_pair', 'score'],
      dtype='object')

In [25]:
# Initialize tqdm for the groupby process
groupby_pbar = tqdm(total=len(df2['postal_code'].unique()) * len(df2['category'].unique()), desc="Grouping DataFrames")

# Select top 3 and bottom 3 rows by score within each group
result_dfs = []
for name_2, group in df2.groupby(['postal_code', 'category']):
    top_rows = group.nlargest(10, 'score')
    bottom_rows = group.nsmallest(10, 'score')
    result_dfs.extend([top_rows, bottom_rows])
    groupby_pbar.update(1)



Grouping DataFrames:  86%|██████████▎ | 313/366 [12:46<02:09,  2.45s/it]

Grouping DataFrames:  86%|█████████▍ | 313/366 [00:31<00:00, 117.03it/s]

In [26]:
result_df = pd.concat(result_dfs)

In [28]:
result_df.to_csv('NLP_VADER_sent_3grams_by_zipANDcat_top10.csv')

In [29]:
result_df.head()

,postal_code,category,business_id,preprocessed_text_v1,num_tokens,avg_sent_across_3gram_VADER,ngram_pair,score
6958130,19019,beauty,JHmif2Bu9nmRY_-wBptbXg,cant begin describe impressed michele makeup h...,59,0.168465,"('rescue', 'super', 'safe')",0.8779
6957986,19019,beauty,JHmif2Bu9nmRY_-wBptbXg,would like opportunity personally thank michel...,77,0.302777,"('grateful', 'appreciative', 'honesty')",0.8689
6957987,19019,beauty,JHmif2Bu9nmRY_-wBptbXg,would like opportunity personally thank michel...,77,0.302777,"('appreciative', 'honesty', 'kindness')",0.8689
6958111,19019,beauty,JHmif2Bu9nmRY_-wBptbXg,michele fabulous wonderful job indian wedding ...,37,0.259354,"('appreciate', 'easy', 'friendly')",0.8316
6957991,19019,beauty,JHmif2Bu9nmRY_-wBptbXg,would like opportunity personally thank michel...,77,0.302777,"('area', 'beautiful', 'amazing')",0.8271


In [30]:
result_df[result_df['score'] < 0]

,postal_code,category,business_id,preprocessed_text_v1,num_tokens,avg_sent_across_3gram_VADER,ngram_pair,score
6958034,19019,beauty,JHmif2Bu9nmRY_-wBptbXg,amazing makeup artisthairstylist wedding day n...,46,0.266820,"('her', 'bad', 'miving')",-0.5423
6958035,19019,beauty,JHmif2Bu9nmRY_-wBptbXg,amazing makeup artisthairstylist wedding day n...,46,0.266820,"('bad', 'miving', 'sf')",-0.5423
6958008,19019,beauty,JHmif2Bu9nmRY_-wBptbXg,amazing makeup artisthairstylist wedding day n...,46,0.266820,"('never', 'felt', 'beautiful')",-0.4847
6957983,19019,beauty,JHmif2Bu9nmRY_-wBptbXg,would like opportunity personally thank michel...,77,0.302777,"('getting', 'dangerous', 'situation')",-0.4767
6958039,19019,beauty,JHmif2Bu9nmRY_-wBptbXg,contacted michele month wedding since plan cha...,40,0.175639,"('email', 'quickly', 'stress')",-0.4215
...,...,...,...,...,...,...,...,...
14594169,19195,shopping,gN-w5zqrN0-CNPy0H7BWwQ,absolutely love store go to gift want somethin...,100,0.117946,"('opinion', 'terrible', 'shame')",-0.7351
14594170,19195,shopping,gN-w5zqrN0-CNPy0H7BWwQ,absolutely love store go to gift want somethin...,100,0.117946,"('terrible', 'shame', 'usually')",-0.7351
14593223,19195,shopping,gN-w5zqrN0-CNPy0H7BWwQ,city kill time stopped here store adorable man...,63,0.132033,"('city', 'kill', 'time')",-0.6908
14594055,19195,shopping,gN-w5zqrN0-CNPy0H7BWwQ,love wandering open house something everyone t...,83,0.075635,"('go', 'go', 'kill')",-0.6908
